# Here we will discuss following topics :
1. Method to create dataframes
2. Adding monotonically increasing id
3. Use CASE WHEN THEN Transformation
4. Casting your columns
5. Adding columns to Dataframes
6. Dropping columns
7. Dropping duplicate rows
8. Sorting Dataframes

In [38]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id, when, expr
from pyspark.sql.types import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder \
                    .appName("Spark Exploring Columns Demo") \
                    .master("local[3]") \
                    .enableHiveSupport() \
                    .getOrCreate()

# 1. Method to create dataframes


In [39]:
data_list = [("Ravi", "28", "1", "2002"),
                 ("Abd", "23", "5", "81"),  # 1981
                 ("John", "12", "12", "6"),  # 2006
                 ("Rosy", "7", "8", "63"),  # 1963
                 ("Abd", "23", "5", "81")]  # 1981
raw_df = spark.createDataFrame(data_list)
raw_df.show()                 

+----+---+---+----+
|  _1| _2| _3|  _4|
+----+---+---+----+
|Ravi| 28|  1|2002|
| Abd| 23|  5|  81|
|John| 12| 12|   6|
|Rosy|  7|  8|  63|
| Abd| 23|  5|  81|
+----+---+---+----+



In [40]:
# to assign menigful names to the columns
raw_df = spark.createDataFrame(data_list).toDF("name" , "day", "month", "year").repartition(3)
raw_df.show()


+----+---+-----+----+
|name|day|month|year|
+----+---+-----+----+
|Ravi| 28|    1|2002|
| Abd| 23|    5|  81|
| Abd| 23|    5|  81|
|John| 12|   12|   6|
|Rosy|  7|    8|  63|
+----+---+-----+----+



#  2. Adding monotonically increasing id

In [41]:
df1 = raw_df.withColumn("id", monotonically_increasing_id())
df1.show()

+----+---+-----+----+-----------+
|name|day|month|year|         id|
+----+---+-----+----+-----------+
|Ravi| 28|    1|2002|          0|
| Abd| 23|    5|  81|          1|
| Abd| 23|    5|  81| 8589934592|
|John| 12|   12|   6|17179869184|
|Rosy|  7|    8|  63|17179869185|
+----+---+-----+----+-----------+



# 3. Use CASE WHEN THEN Transformation


In [42]:
df2 = df1.withColumn("year", expr("""
case when year <21 then year + 2000
     when year <100 then year + 1900
     else year                           
end """
))
df2.show()
df2.printSchema()

+----+---+-----+------+-----------+
|name|day|month|  year|         id|
+----+---+-----+------+-----------+
|Ravi| 28|    1|  2002|          0|
| Abd| 23|    5|1981.0|          1|
| Abd| 23|    5|1981.0| 8589934592|
|John| 12|   12|2006.0|17179869184|
|Rosy|  7|    8|1963.0|17179869185|
+----+---+-----+------+-----------+

root
 |-- name: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- id: long (nullable = false)



# 4. Casting your columns


In [43]:
# inline cast
df2 = df1.withColumn("year", expr("""
case when year <21 then cast(year as int) + 2000
     when year <100 then cast(year as int) + 1900
     else year                           
end """
))
df2.show()
df2.printSchema()

+----+---+-----+----+-----------+
|name|day|month|year|         id|
+----+---+-----+----+-----------+
|Ravi| 28|    1|2002|          0|
| Abd| 23|    5|1981|          1|
| Abd| 23|    5|1981| 8589934592|
|John| 12|   12|2006|17179869184|
|Rosy|  7|    8|1963|17179869185|
+----+---+-----+----+-----------+

root
 |-- name: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- id: long (nullable = false)



In [44]:
# change the schema
df2 = df1.withColumn("year", expr("""
case when year <21 then year + 2000
     when year <100 then year + 1900
     else year                           
end """
).cast(IntegerType()))
df2.show()
df2.printSchema()

+----+---+-----+----+-----------+
|name|day|month|year|         id|
+----+---+-----+----+-----------+
|Ravi| 28|    1|2002|          0|
| Abd| 23|    5|1981|          1|
| Abd| 23|    5|1981| 8589934592|
|John| 12|   12|2006|17179869184|
|Rosy|  7|    8|1963|17179869185|
+----+---+-----+----+-----------+

root
 |-- name: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- id: long (nullable = false)



In [45]:
# changing the type from begining 
df1.show()
df1.printSchema()
df3 = df1.withColumn("day", col("day").cast(IntegerType())) 
df3 = df1.withColumn("month", col("month").cast(IntegerType())) 
df3 = df1.withColumn("year", col("year").cast(IntegerType()))
df3.printSchema()

df4 = df3.withColumn("year", expr("""
case when year <21 then year + 2000
     when year <100 then year + 1900
     else year                           
end """
))
df4.show()
df4.printSchema()


+----+---+-----+----+-----------+
|name|day|month|year|         id|
+----+---+-----+----+-----------+
|Ravi| 28|    1|2002|          0|
| Abd| 23|    5|  81|          1|
| Abd| 23|    5|  81| 8589934592|
|John| 12|   12|   6|17179869184|
|Rosy|  7|    8|  63|17179869185|
+----+---+-----+----+-----------+

root
 |-- name: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- id: long (nullable = false)

root
 |-- name: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- id: long (nullable = false)

+----+---+-----+----+-----------+
|name|day|month|year|         id|
+----+---+-----+----+-----------+
|Ravi| 28|    1|2002|          0|
| Abd| 23|    5|1981|          1|
| Abd| 23|    5|1981| 8589934592|
|John| 12|   12|2006|17179869184|
|Rosy|  7|    8|1963|17179869185|
+----+---+-----+----+-----------+

root
 |-- name: str

In [46]:
# column object expression for case expression
df5 = df4.withColumn("year", \
                     when(col("year") < 21, col("year")+ 2000) \
                     .when(col("year") < 100, col("year")+ 1900) \
                     .otherwise(col("year"))
                     )
df5.show()

+----+---+-----+----+-----------+
|name|day|month|year|         id|
+----+---+-----+----+-----------+
|Ravi| 28|    1|2002|          0|
| Abd| 23|    5|1981|          1|
| Abd| 23|    5|1981| 8589934592|
|John| 12|   12|2006|17179869184|
|Rosy|  7|    8|1963|17179869185|
+----+---+-----+----+-----------+



# 5. Adding/Remove columns columns to Dataframes


In [47]:
# add column by combining day, month and year
# using sql expression 

df6 = df5.withColumn("dob", expr("to_date(concat(day, '/', month, '/', year), 'd/M/y')"))
df6.show()

+----+---+-----+----+-----------+----------+
|name|day|month|year|         id|       dob|
+----+---+-----+----+-----------+----------+
|Ravi| 28|    1|2002|          0|2002-01-28|
| Abd| 23|    5|1981|          1|1981-05-23|
| Abd| 23|    5|1981| 8589934592|1981-05-23|
|John| 12|   12|2006|17179869184|2006-12-12|
|Rosy|  7|    8|1963|17179869185|1963-08-07|
+----+---+-----+----+-----------+----------+



In [48]:
# using column expression
df7 = df5.withColumn("dob", to_date(expr("concat(day, '/', month, '/', year)"), 'd/M/y'))
df7.show()

+----+---+-----+----+-----------+----------+
|name|day|month|year|         id|       dob|
+----+---+-----+----+-----------+----------+
|Ravi| 28|    1|2002|          0|2002-01-28|
| Abd| 23|    5|1981|          1|1981-05-23|
| Abd| 23|    5|1981| 8589934592|1981-05-23|
|John| 12|   12|2006|17179869184|2006-12-12|
|Rosy|  7|    8|1963|17179869185|1963-08-07|
+----+---+-----+----+-----------+----------+



# 6. Dropping columns


In [49]:
# removing a column
df8 = df7.drop("day", "month", "year")
df8.show()

+----+-----------+----------+
|name|         id|       dob|
+----+-----------+----------+
|Ravi|          0|2002-01-28|
| Abd|          1|1981-05-23|
| Abd| 8589934592|1981-05-23|
|John|17179869184|2006-12-12|
|Rosy|17179869185|1963-08-07|
+----+-----------+----------+



# 7. Dropping duplicate rows


In [50]:
df9 = df8.dropDuplicates(["name", "dob"])
df9.show()

+----+-----------+----------+
|name|         id|       dob|
+----+-----------+----------+
| Abd|          1|1981-05-23|
|Ravi|          0|2002-01-28|
|Rosy|17179869185|1963-08-07|
|John|17179869184|2006-12-12|
+----+-----------+----------+



# 8. Sorting Dataframes

In [51]:
df10 = df9.sort(expr("dob desc"))

df10.show()

+----+-----------+----------+
|name|         id|       dob|
+----+-----------+----------+
|Rosy|17179869185|1963-08-07|
| Abd|          1|1981-05-23|
|Ravi|          0|2002-01-28|
|John|17179869184|2006-12-12|
+----+-----------+----------+

